# Création d'un chatbot personalisé avec RAG
Ce chatbot a été créé pour une entreprise fictive de vente de vêtements. L'entreprise s'appelle Gora et son chatbot a été développé pour aider ses clients et ses employés. Le chatbot est basé sur la technologie LLM avec le renfort de RAG (Retrieval-augmented generation).

La RAG est une technique permettant d'améliorer la précision et la fiabilité des modèles génératifs d'IA à l'aide de données obtenues à partir de sources externes.

Des documents ont été créés, comprenant des données sur les produits, les stocks et une série de questions fréquemment posées, y compris des informations générales sur l'entreprise, son fonctionnement, les conditions générales d'utilisation et plus encore, afin que le LLM puisse répondre avec précision.

Le chatbot fonctionne sur une interface générée par Gradio.

###  Installation des dépendances et import des bibliothèques et configuration des paramètres

Notes : La cellule suivante est mise _en commentaires_ pour éviter son exécution. Si ce code est hébergé sur internet, il faut créer le fichier "requirements.txt" et l'héberger dans le même dossier. Dans ce fichier, il doit y avoir les versions de chaque paquet.

In [1]:
# Liste de tous les paquets installés et de leurs versions
#!pip freeze

# Enregistrer la liste dans un fichier requirements.txt
#!pip freeze > requirements.txt

In [2]:
!pip install llama-index #== version
!pip install llama-index-embeddings-huggingface
!pip install peft #Parameter efficient fine-tuning
!pip install auto-gptq # Ouvrir/télécharger le modele GPTQ quantized
!pip install bitsandbytes #Pour utiliser le model quantizised
!pip install transformers
!pip install optimum
!pip install pipeline
!pip install --upgrade llama_index
!pip install --upgrade openai
!pip install pandas openpyxl # Instalation des packs pour lire les docs
!pip install gradio #Interface du chat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_n

In [3]:
import traceback
import pandas as pd
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

from llama_index.embeddings.huggingface import HuggingFaceEmbedding # Transforme les mots en vecteur
from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


### Configuration du modèle d'embedding et création de la base de connaisances

Le modele d'embedding a été pris de Hugging Face Hub. Il sert à vectoriser les mots.

In [4]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [5]:
# Variables vers le chemin des fichiers
faq_file = "FAQ.xlsx"
stock_file = "Stock.xlsx"
product_info_file = "InfoProduits.xlsx"

# Lecture des fichiers Excel
faq_data = pd.read_excel(faq_file, engine='openpyxl')
stock_data = pd.read_excel(stock_file, engine='openpyxl')
product_info_data = pd.read_excel(product_info_file, engine='openpyxl')

# Création du objet Document
documents = []

# Initialiser la liste `documents` avec la structure correcte
for _, row in faq_data.iterrows():
    faq = f"Q : {row['Question']}\nA : {row['Answer']}"
    documents.append(Document(text=faq))

# Initialiser `Document` pour les données du stock
for _, row in stock_data.iterrows():
    stock_info = f"Produit : {row['Product Name']}, Stock : {row['Stock Quantity']}"
    documents.append(Document(text=stock_info))

# Initialiser `Document` pour les informations produit
for _, row in product_info_data.iterrows():
    product_info = (
        f"Code produit : {row['Product Code']}, "
        f"Nom du produit : {row['Product Name']}, "
        f"Description : {row['Description']}, "
        f"Caractéristiques : {row['Characteristics']}, "
        f"Taille : {row['Size']}, "
        f"Prix : {row['Price (€)']}€"
    )
    documents.append(Document(text=product_info))

In [17]:
#Check du documents
documents[:5]

[Document(id_='891a369b-fa03-4cdf-aeef-0253e6253d97', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Q : 1. Quels sont les horaires d'ouverture du magasin ?\nA : Notre magasin est ouvert de 9h à 21h, du lundi au samedi, et de 10h à 20h le dimanche.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='19bf63a1-9135-418b-8851-c927cecbb018', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Q : 2. Où se trouvent vos magasins ?\nA : Vous pouvez trouver nos magasins dans les principaux centres commerciaux et rues commerçantes du pays. Consultez le localisateur de magasins sur notre site Web pour trouver le plus proche.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadat

### Création de l'index de la base de données et du moteur de requête

In [7]:
# Créer un index à partir des documents
index = VectorStoreIndex.from_documents(documents)

# Créer le moteur de requête
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3   #Il récupère les  3 resultats les plus importants pqr rapport au query
    )
#Montage du query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

#### Exemple de requête pour récupérer des documents pertinents (RAG) Generation d'une réponse

In [13]:
query = "Quels sont vos horaires ?"
response = query_engine.query(query)

# Reformater la réponse pour présenter des informations pertinentes
contexte = "Contexte :\n"
for node in response.source_nodes:
    contexte += node.text + "\n -Iagobot \n\n"

print(contexte)

Contexte :
Q : 1. Quels sont les horaires d'ouverture du magasin ?
A : Notre magasin est ouvert de 9h à 21h, du lundi au samedi, et de 10h à 20h le dimanche.
 -Iagobot 

Q : 90. Offrez-vous des services de livraison le même jour ?
A : Nous proposons des services de livraison le même jour dans certaines zones. Consultez notre site Web pour voir si votre emplacement est éligible.
 -Iagobot 

Q : 55. Quelle est votre politique concernant les échanges de tailles ?
A : Nous offrons des échanges de tailles dans les 30 jours suivant l'achat. Contactez le service client pour organiser un échange.
 -Iagobot 




### Chargement du modèle quantifié et du tokenizer

In [9]:
# Charger le modèle quantifié (Quantized) et tokenizer
model_name = "TheBloke/Mistral-7B-Claude-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

### Définition de la fonction de génération de réponse

In [10]:
# Prompt du système pour IagoBOT
system_prompt = (
    "IagoBOT est un assistant pour répondre aux questions des clients, il travaille chez Gora, une marque de mode. "
    "Il communique dans un langage clair et accessible, rendant l'interaction naturelle et engageante. "
    "si nécessaire, utiliser le contexte suivant pour répondre à la question de l'utilisateur et répondre de la manière la plus appropriée"
)

def generate_response(user_input, history, system_prompt, max_new_tokens=256, temperature=0.5):
    # Récupérer le contexte des documents
    response = query_engine.query(user_input)
    context = "Contexte :\n"
    for node in response.source_nodes:
        context += node.text + "\n"

    # Créer le prompt avec le contexte récupéré et l'historique de la conversation
    prompt = f"{context}\n"
    for user, assistant in history:
        prompt += f"{user}\n{assistant}\n"
    prompt += f"UTILISATEUR: {user_input}\nIagoBOT:"

    # Ajouter le message du système au début de la conversation, une seule fois
    if not history:
        prompt = f"{system_prompt}\n{prompt}"

    # Tokeniser le prompt
    inputs = tokenizer(prompt, return_tensors='pt', padding=True).to('cuda')

    # Générer la réponse
    output = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    # Décoder la réponse générée
    response_text = tokenizer.decode(output[0], skip_special_tokens=True).split("IagoBOT:")[-1].strip()

    # Mettre à jour l'historique avec la réponse de l'assistant
    history.append((f"UTILISATEUR: {user_input}", f"IagoBOT: {response_text}"))

    return history, history


### Interface du chat.

In [12]:
# Interface Gradio
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="IAGOBot")
    state = gr.State([])  # État initial pour l'historique de la conversation

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(placeholder="Posez vos questions à IAgoBot...")
            send_button = gr.Button("Envoyer")

    def user_submit(input, history):
        updated_history, response = generate_response(input, history, system_prompt)
        return updated_history, updated_history

    send_button.click(user_submit, inputs=[user_input, state], outputs=[chatbot, state])

demo.launch(share=True, debug = True )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a32f5c141ed6ad38d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1d698953888a40e479.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://a32f5c141ed6ad38d9.gradio.live
